In [1]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
dpath = "/content/drive/My Drive/LAB/data/"
files = os.listdir(dpath)
files.sort()
print(files)

['ADANIPORTS.csv', 'ASIANPAINT.csv', 'AXISBANK.csv', 'BAJAJ-AUTO.csv', 'BAJAJFINSV.csv', 'BAJFINANCE.csv', 'BHARTIARTL.csv', 'BPCL.csv', 'BRITANNIA.csv', 'CIPLA.csv', 'COALINDIA.csv', 'DRREDDY.csv', 'EICHERMOT.csv', 'GAIL.csv', 'GRASIM.csv', 'HCLTECH.csv', 'HDFC.csv', 'HDFCBANK.csv', 'HEROMOTOCO.csv', 'HINDALCO.csv', 'HINDUNILVR.csv', 'ICICIBANK.csv', 'INDUSINDBK.csv', 'INFY.csv', 'IOC.csv', 'ITC.csv', 'JSWSTEEL.csv', 'KOTAKBANK.csv', 'LT.csv', 'MARUTI.csv', 'MM.csv', 'NESTLEIND.csv', 'NTPC.csv', 'ONGC.csv', 'POWERGRID.csv', 'RELIANCE.csv', 'SBIN.csv', 'SHREECEM.csv', 'SUNPHARMA.csv', 'TATAMOTORS.csv', 'TATASTEEL.csv', 'TCS.csv', 'TECHM.csv', 'TITAN.csv', 'ULTRACEMCO.csv', 'UPL.csv', 'VEDL.csv', 'WIPRO.csv', 'ZEEL.csv']


In [4]:
df = pd.read_csv(dpath + files[0]).set_index('Date')
df = df[['Close']]

In [5]:
scaler = MinMaxScaler(feature_range=(-1, 1))
df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))

In [6]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

look_back = 30 # choose sequence length
x_train, y_train, x_test, y_test = load_data(df, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (2634, 29, 1)
y_train.shape =  (2634, 1)
x_test.shape =  (658, 29, 1)
y_test.shape =  (658, 1)


In [7]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [8]:
input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

LSTM(
  (lstm): LSTM(1, 30, num_layers=5, batch_first=True)
  (fc): Linear(in_features=30, out_features=1, bias=True)
)
22
torch.Size([120, 1])
torch.Size([120, 30])
torch.Size([120])
torch.Size([120])
torch.Size([120, 30])
torch.Size([120, 30])
torch.Size([120])
torch.Size([120])
torch.Size([120, 30])
torch.Size([120, 30])
torch.Size([120])
torch.Size([120])
torch.Size([120, 30])
torch.Size([120, 30])
torch.Size([120])
torch.Size([120])
torch.Size([120, 30])
torch.Size([120, 30])
torch.Size([120])
torch.Size([120])
torch.Size([1, 30])
torch.Size([1])


In [9]:
from sklearn.metrics import mean_absolute_error

In [ ]:
files = os.listdir(dpath)
files.sort()
log = dict()
mpath = '/content/drive/My Drive/LAB/output_lstm/'
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
num_epochs = 100
optimiser = torch.optim.Adam(model.parameters(), lr=0.001)

rmse = []
maes = []

from tqdm import tqdm, trange

for t in trange(num_epochs):
    for file in files:
        try:
            df = pd.read_csv(dpath + file).set_index('Date')
            df = df[['Close']]
            scaler = MinMaxScaler(feature_range=(-1, 1))
            df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
            look_back = 30 # choose sequence length
            x_train, y_train, x_test, y_test = load_data(df, look_back)

            x_train = torch.from_numpy(x_train).type(torch.Tensor)
            x_test = torch.from_numpy(x_test).type(torch.Tensor)
            y_train = torch.from_numpy(y_train).type(torch.Tensor)
            y_test = torch.from_numpy(y_test).type(torch.Tensor)

            loss_fn = torch.nn.MSELoss()
            hist = np.zeros(num_epochs)
            seq_dim =look_back-1

            y_train_pred = model(x_train)

            loss = loss_fn(y_train_pred, y_train)
            print(f"{file}: Epoch ", t, "MSE: ", loss.item(), end = '\r')
            hist[t] = loss.item()

            # Zero out gradient, else they will accumulate between epochs
            optimiser.zero_grad()

            # Backward pass
            loss.backward()

            # Update parameters
            optimiser.step()

            y_test_pred = model(x_test)

            # invert predictions
            if t == num_epochs - 1:
                y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
                y_train = scaler.inverse_transform(y_train.detach().numpy())
                y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
                y_test = scaler.inverse_transform(y_test.detach().numpy())


                mae = mean_absolute_error(y_test, y_test_pred)
                maes.append(mae)
                rmse.append(sqrt(mean_squared_error(y_test, y_test_pred)))

        except:
            log[file] = 'ERROR'

torch.save(model.state_dict(), mpath + 'global_lstm' + '.pth')

In [ ]:
import json
for key, val in log.items():
    log[key] = str(val)
with open(mpath + 'log.json', 'w') as json_file:
    json.dump(log, json_file)

5555

In [10]:
import numpy as np
import random
import pandas as pd

import datetime
import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable

import matplotlib.pyplot as plt
import os

In [11]:
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []

    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back):
        data.append(data_raw[index: index + look_back])

    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);

    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]

    x_test = data[train_set_size:,:-1]
    y_test = data[train_set_size:,-1,:]

    return [x_train, y_train, x_test, y_test]

In [12]:
def MAPE(Y_Predicted,Y_actual):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

input_dim = 1
hidden_dim = 30
num_layers = 5
output_dim = 1
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

import matplotlib.dates as mdates

model.load_state_dict(torch.load('/content/drive/My Drive/LAB/output_lstm/global_lstm.pth'))
model.eval()

log_hist = ""


rmse = []
maes = []

def infer(file):
    global log_hist
    dpath = "/content/drive/My Drive/LAB/data/"
    mpath = '/content/drive/My Drive/LAB/output_lstm/'
    df = pd.read_csv(dpath + file).set_index('Date')
    df.index = pd.to_datetime(df.index)
    df = df[['Close']]
    scaler = MinMaxScaler(feature_range=(-1, 1))
    df['Close'] = scaler.fit_transform(df['Close'].values.reshape(-1,1))
    look_back = 30 # choose sequence length
    x_train, y_train, x_test, y_test = load_data(df, look_back)
    x_train = torch.from_numpy(x_train).type(torch.Tensor)
    x_test = torch.from_numpy(x_test).type(torch.Tensor)
    y_train = torch.from_numpy(y_train).type(torch.Tensor)
    y_test = torch.from_numpy(y_test).type(torch.Tensor)
#     print(x_train, x_train.shape)
    with torch.no_grad():
        y_train_pred = model(x_train)
        y_test_pred = model(x_test)
    y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
    y_train = scaler.inverse_transform(y_train.detach().numpy())
    y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
    y_test = scaler.inverse_transform(y_test.detach().numpy())

    mae = mean_absolute_error(y_test, y_test_pred)
    maes.append(mae)
    rmse.append(sqrt(mean_squared_error(y_test, y_test_pred)))

    err_test, err_train = MAPE(y_test_pred, y_test), MAPE(y_train_pred, y_train)

    logln = f"{file.split()[0]} had {err_train}% train error and {err_test}% test error\n"
    # print(logln)
    log_hist += logln

    figure, axes = plt.subplots(figsize=(15, 6))
    axes.xaxis.set_major_locator(mdates.AutoDateLocator())
    axes.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))

    axes.plot(df[len(df)-len(y_test):].index, y_test, color = 'red', label = 'Real Stock Price')
    axes.plot(df[len(df)-len(y_test):].index, y_test_pred, color = 'blue', label = 'Predicted Stock Price')
    #axes.xticks(np.arange(0,394,50))
    plt.title(f'{file} Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('Stock Price')
    plt.legend()
    plt.savefig('/content/drive/My Drive/LAB/output_lstm/' + file.split('.')[0] + '.png')
    plt.show()

RuntimeError: Error(s) in loading state_dict for LSTM:
	size mismatch for lstm.weight_ih_l0: copying a param with shape torch.Size([160, 1]) from checkpoint, the shape in current model is torch.Size([120, 1]).
	size mismatch for lstm.weight_hh_l0: copying a param with shape torch.Size([160, 40]) from checkpoint, the shape in current model is torch.Size([120, 30]).
	size mismatch for lstm.bias_ih_l0: copying a param with shape torch.Size([160]) from checkpoint, the shape in current model is torch.Size([120]).
	size mismatch for lstm.bias_hh_l0: copying a param with shape torch.Size([160]) from checkpoint, the shape in current model is torch.Size([120]).
	size mismatch for lstm.weight_ih_l1: copying a param with shape torch.Size([160, 40]) from checkpoint, the shape in current model is torch.Size([120, 30]).
	size mismatch for lstm.weight_hh_l1: copying a param with shape torch.Size([160, 40]) from checkpoint, the shape in current model is torch.Size([120, 30]).
	size mismatch for lstm.b

In [ ]:
for file in tqdm(files):
    try:
        infer(file)
    except:
        continue

In [ ]:
with open('/content/drive/My Drive/LAB/output_lstm/LOG.log', 'w+') as f:
    f.write(log_hist)
f.close()

In [ ]:
print(rmse)
print(maes)